# 📊 RAGAS Evaluation for Index Advisor Crew

This notebook is designed to evaluate the output quality of your CrewAI-based Index Advisor using RAGAS metrics.

You can run multiple scenarios based on different timestamps, tickers, and market conditions.

In [1]:
# ✅ Install dependencies if needed
!pip install ragas pandas openai datasets


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# ✅ Import libraries
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision
import pandas as pd
from datasets import Dataset

c:\Users\Jovan\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
from ragas.llms import LangchainLLMWrapper
from langchain_openai import ChatOpenAI
from langchain_core.messages import BaseMessage
from langchain_core.language_models import BaseLanguageModel
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

langchain_openai_model: BaseLanguageModel[BaseMessage] = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, openai_api_key=os.getenv("OPENAI_API_KEY"))

evaluator_llm = LangchainLLMWrapper(langchain_llm=langchain_openai_model)

In [8]:
from langchain_openai import OpenAIEmbeddings
from ragas.embeddings import LangchainEmbeddingsWrapper

openai_embeddings = OpenAIEmbeddings(api_key=api_key)

evaluator_embeddings = LangchainEmbeddingsWrapper(openai_embeddings)

In [ ]:
# ✅ Define your evaluation scenarios
# Fill this in with the actual results from your CrewAI runs

evaluation_data = [
    {
        "question": "What is the technical outlook for ^GSPC as of June 2025?",
        "context": "SMA_20: 6169.99, RSI_14: 72.33, Close: 6263.7",
        "answer": "The decision to KEEP a position in the S&P 500 (^GSPC) reflects a"
        " balanced view of strong earnings, stable macroeconomic indicators, and encouraging technical trends,"
        " such as the index trading above its 20-day SMA. However, elevated RSI levels"
        " signal potential short-term corrections, while persistent inflation, high interest rates, "
        "and global uncertainties introduce downside risks. Given this mix of optimism and caution, "
        "holding the position allows investors to benefit from current momentum while staying flexible amid ongoing volatility.",
        "ground_truth": "The S&P 500 shows upward momentum, though RSI levels suggest caution due to overbought territory."
    },
    # Add more scenarios below
]

dataset = Dataset.from_list(evaluation_data)

In [ ]:

def load_md_file(filepath):
    """Reads a markdown file and splits it into paragraphs or context chunks."""
    with open(filepath, "r", encoding="utf-8") as f:
        content = f.read()
    return [chunk.strip() for chunk in content.split("\n\n") if chunk.strip()]

# Load both .md files
context_file_1 = load_md_file("output_news_task.md")
context_file_2 = load_md_file("outout_teknikal.md")

# Combine both retrieved contexts
retrieved_contexts = context_file_1 + context_file_2

print(retrieved_contexts)

FileNotFoundError: [Errno 2] No such file or directory: 'output_news_task.md'

In [ ]:
from ragas.metrics import Faithfulness,ResponseRelevancy,LLMContextPrecisionWithoutReference,LLMContextRecall,NoiseSensitivity
from ragas.dataset_schema import SingleTurnSample

sample = SingleTurnSample(
    user_input="is the SNP500(^GSPC) good to buy?",
    response="The decision to KEEP a position in the S&P 500 (^GSPC) reflects a balanced view of strong earnings, stable macroeconomic indicators, and encouraging technical trends, such as the index trading above its 20-day SMA. However, elevated RSI levels signal potential short-term corrections, while persistent inflation, high interest rates, and global uncertainties introduce downside risks. Given this mix of optimism and caution, holding the position allows investors to benefit from current momentum while staying flexible amid ongoing volatility.",
    reference="Sertraline adalah obat antidepresan golongan Selective Serotonin Reuptake Inhibitor (SSRI) yang bekerja dengan cara menyeimbangkan kadar serotonin dalam otak, sehingga membantu mengatur suasana hati. Obat ini umum digunakan untuk mengatasi depresi, gangguan obsesif kompulsif (OCD), gangguan panik, PTSD, dan gangguan kecemasan sosial. Dosis awal untuk dewasa umumnya 50 mg sekali sehari, yang bisa ditingkatkan bertahap hingga maksimal 200 mg per hari tergantung kondisi dan respon pasien. Efek samping yang mungkin timbul meliputi kesulitan tidur, kelelahan, dan gangguan konsentrasi. Penggunaan Sertraline perlu dihindari jika sedang menggunakan MAOI, memiliki riwayat gangguan bipolar, penyakit jantung, atau sedang hamil dan menyusui, kecuali atas rekomendasi dokter. Sertraline juga tidak disarankan untuk dikonsumsi bersama alkohol dan harus digunakan dengan pengawasan medis. Beberapa merek dagang dari Sertraline termasuk Zoloft, Zerlin, dan Serlof. Obat ini tersedia dalam bentuk tablet dan hanya boleh digunakan sesuai resep dokter.",
    retrieved_contexts = retrieved_contexts
  )
scorer = ResponseRelevancy(llm=evaluator_llm, embeddings=evaluator_embeddings)
RR_score = scorer.single_turn_score(sample)  
print("Response Relevancy Score:", RR_score)